In [1]:
import pandas as pd
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt

credentials_path = './pamf-dwh-775beaae50ff.json'
project_id = 'pamf-dwh'
sql = ''' SELECT DATE(disbursement_date) as disbursement_date, 
                 loloanID,
                 account_Number,
                 rContactID,
                 Status,
                 loan_cycle,
                 loApplicationID
        FROM `pamf-dwh.DFS_VIEW.deboursement`  ''' 
deboursement =pd.read_gbq( sql, project_id, private_key=credentials_path, dialect='standard')

sql = '''SELECT DATE(TransactionDate) AS TransactionDate,
            loLoanID,
            rContactID,
            admin_paid,
            principal_paid,
            penalty_paid
        FROM `pamf-dwh.DFS_VIEW.repayement`'''
repayement = pd.read_gbq(sql, project_id, private_key = credentials_path, dialect = 'standard')

#merging repayment and debouresment
merged = repayement.merge(deboursement, left_on=['loLoanID','rContactID'], right_on = ['loloanID','rContactID'])
merged['delta_date'] = merged['TransactionDate'] - merged['disbursement_date']
merged['delta_date'] = merged['delta_date'].apply(lambda x: x.days)
merged['day_late'] = merged['delta_date'] - 30
merged['writeoff'] = merged['day_late'] >120

#getting all the clients with days late > 50 from 2018
test = merged[(merged['day_late']>50) ]
test.groupby('TransactionDate')['principal_paid'].sum().to_excel('payment_general_trend.xlsx')

#getting the clients from the beginning of the campaign only with 50 days late
test_plus_50 = merged[(merged['day_late']>=50) & (merged['TransactionDate']>= '2019-05-22') ]
test_plus_50.groupby('TransactionDate')['principal_paid'].sum().to_excel('campagne_payment_general_trend.xlsx')
test_plus_50=test_plus_50.fillna(0)
test_plus_50['Total_paid'] = test_plus_50['admin_paid'] + test_plus_50['penalty_paid'] + test_plus_50['principal_paid'] 


#separating if they are simply late or writeoff
#simply late
exporting_file_simple = test_plus_50[test_plus_50['writeoff'] == False ].groupby(['TransactionDate'])[['Total_paid']].sum()
exporting_file_simple.rename(columns={"Total_paid": "50-120j_retard_paid"},inplace=True)
#writeoff
exporting_file_writeoff = test_plus_50[test_plus_50['writeoff'] == True ].groupby(['TransactionDate'])[['Total_paid']].sum()
exporting_file_writeoff.rename(columns={"Total_paid": "writeoff_paid"}, inplace=True)

#exporting the file to excel
exporting_file = exporting_file_simple.join(exporting_file_writeoff)
exporting_file['Total_paid']= exporting_file['50-120j_retard_paid']+exporting_file['writeoff_paid']
exporting_file.to_excel('Impact_campagne_Alefa.xlsx')


E:\Anaconda3\lib\site-packages\pandas\io\gbq.py:149: FutureWarning: private_key is deprecated and will be removed in a future version.Use the credentials argument instead. See https://pandas-gbq.readthedocs.io/en/latest/howto/authentication.html for examples on using the credentials argument with service account keys.
  credentials=credentials, verbose=verbose, private_key=private_key)


In [6]:
test_plus_50.dtypes


TransactionDate      datetime64[ns]
loLoanID                      int64
rContactID                    int64
admin_paid                  float64
principal_paid              float64
penalty_paid                float64
disbursement_date    datetime64[ns]
loloanID                      int64
account_Number               object
Status                        int64
loan_cycle                    int64
loApplicationID               int64
delta_date                    int64
day_late                      int64
writeoff                       bool
Total_paid                  float64
dtype: object

In [2]:
test.principal_paid.sum()

534476500.0

In [3]:
# campagne repayment so far: 
test_plus_50.principal_paid.sum()

54559850.0

In [ ]:
#exporting_file
test_plus_50.head(2)

In [ ]:
test_plus_50.head(2)

In [ ]:

exporting_file_simple.head(2)

In [4]:
exporting_file.Total_paid.sum()

60494926.0